In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!conda install -y gdown

In [ ]:
#!gdown --id 1bq3IbdKNSSIAhR3JXJgDXh5XgHbvuyAo

In [ ]:
!apt-get install unrar

In [ ]:
! unrar x beedataset.rar /kaggle/working

In [ ]:
import pandas as pd
import numpy as np

def create_label_folder(datafolder):
    
    datacsv = pd.DataFrame(columns = ['filename','label'])

    datalabel = []
    fullname = []

    files = os.listdir(datafolder)
    for images in files:
        if 'bnbee' in images:
            datalabel.append('bumble')
        elif 'hnbee' in images:
            datalabel.append('honey')
        fullname.append(os.path.join(datafolder, images))
    datacsv.label = datalabel
    datacsv.filename = fullname

    datacsv.to_csv('/kaggle/working/beedataset/label.csv', index=False)

    datacsv.head()

In [ ]:
# imprting the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
%matplotlib inline
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow.keras.preprocessing  as kp
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
os.remove('/kaggle/working/beedataset/label2.csv')

In [ ]:
import shutil

In [ ]:
source = r"/kaggle/input/beeclassification/beedataset.rar"
dest = r"/kaggle/working/"



In [ ]:
shutil.copy(source, dest)
print('copied')

In [ ]:
create_label_folder('/kaggle/working/beedataset')

In [ ]:
data = pd.read_csv('/kaggle/working/beedataset/label.csv')
data.head()

In [ ]:

# splitting the dataset
def train_test_valid_split(source_dir, cvs_path,max_batch_size, train_split, test_split, x_col, y_col, class_mode, target_size, scale):

    data=pd.read_csv(cvs_path).copy()

    te_split=test_split/(1-train_split)

    train_df=data.sample(n=None,
                         frac=train_split,
                         replace=False,
                         weights=None,
                         random_state=123,
                         axis=0)

    tr_batch_size= max_batch_size

    #tr_steps=int(len(train_df.index)//tr_batch_size)

    dummy_df=data.drop(train_df.index, axis=0, inplace=False)

    test_df=dummy_df.sample(n=None,
                            frac=te_split,
                            replace=False,
                            weights=None,
                            random_state=123,
                            axis=0)

    #te_batch_size, te_steps=get_bs(len(test_df.index),max_batch_size)
    te_batch_size = max_batch_size
    #te_steps = int(len(test_df.index)//te_batch_size)

    valid_df=dummy_df.drop(test_df.index, axis=0)
    v_batch_size = max_batch_size
    #v_steps=int(len(valid_df.index)//v_batch_size)

    gen=ImageDataGenerator(rescale=scale,
                           rotation_range = rotation_range,
                           width_shift_range = width_shift_range,
                           height_shift_range = height_shift_range,
                           shear_range = shear_range,
                           zoom_range = zoom_range,
                          horizontal_flip = horizontal_flip,
                           fill_mode = fill_mode)

    train_gen=gen.flow_from_dataframe(dataframe=train_df,
                                      directory=source_dir,
                                      batch_size=tr_batch_size,
                                      x_col=x_col,
                                      y_col=y_col,
                                      target_size=target_size,
                                      class_mode=class_mode,
                                      seed=123,
                                      validate_filenames=False)

    test_gen=gen.flow_from_dataframe(dataframe=test_df,
                                     directory=source_dir,
                                     batch_size=te_batch_size,
                                     x_col=x_col,
                                     y_col=y_col,
                                     target_size=target_size,
                                     class_mode=class_mode,
                                     shuffle=False,
                                     validate_filenames=False)

    valid_gen=gen.flow_from_dataframe(dataframe=valid_df,
                                      directory=source_dir,
                                      batch_size=v_batch_size,
                                      x_col=x_col,
                                      y_col=y_col,
                                      target_size=target_size,
                                      class_mode=class_mode,
                                      shuffle=False,
                                      validate_filenames=False)
    tr_steps = len(train_gen)
    te_steps = len(test_gen)
    v_steps = len(valid_gen)

    return train_gen, tr_steps, test_gen, te_steps, valid_gen , v_steps

In [ ]:
# def get_bs(length, b_max):
 #   batch_size=sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=b_max],reverse=True)[0]
  ## return batch_size, steps

# the data augmentation
source_dir = '/kaggle/working/beedataset'
csv_folder = '/kaggle/working/beedataset/label.csv'
train_split = 0.8
test_split = 0.1
x_column = 'filename'
y_column = 'label'
target_size = (260, 260)
scale = 1.0/ 255.0
batch_size = 64
class_mode = 'binary'
rotation_range=20
width_shift_range=0.2
height_shift_range=0.2
shear_range=0.2
zoom_range=0.2
horizontal_flip=True
fill_mode='nearest'
kernel_s=(3,3)


In [ ]:
train_gen, train_steps, test_gen, test_steps, valid_gen, valid_steps=train_test_valid_split(source_dir,
                                                                                            csv_folder,
                                                                                            batch_size,
                                                                                            train_split,
                                                                                            test_split,
                                                                                            x_column,
                                                                                            y_column,
                                                                                            class_mode,
                                                                                            target_size,
                                                                                            scale)
print ('train steps: ', train_steps, '  test steps: ', test_steps, '  valid steps: ', valid_steps)

In [ ]:
train_gen.image_shape